In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import folium

%matplotlib inline
pd.set_option('display.max_columns', 500)

In [ ]:
applications = pd.read_csv('../data/pruned_data.csv', low_memory=False)
applications

In [ ]:
tracts = gpd.read_file('../data/tract.json')
ct_tracts = pd.read_csv('../data/2022tractcrosswalk.csv', usecols=[0,1])
ct_tracts['tract_fips_2020'] = ['0' + str(tract) for tract in ct_tracts['tract_fips_2020']]
ct_tracts['Tract_fips_2022'] = ['0' + str(tract) for tract in ct_tracts['Tract_fips_2022']]
tract_mapping = {row['Tract_fips_2022']:row['tract_fips_2020'] for ind, row in ct_tracts.iterrows()}
tracts['GEOID'] = tracts['GEOID'].replace(tract_mapping)
tracts

In [ ]:
applications['census_string'] = [str(tract)[:-2] for tract in applications['census_tract']]
applications['census_string'] = ['0'+ tract if len(tract) == 10 else tract for tract in applications['census_string']]
applications.head()

In [ ]:
approved = applications[(applications['action_taken'] == 1)]
med_loan_value = approved.groupby('census_string')['loan_amount'].median()
med_loan_value = med_loan_value.reset_index()
med_loan_value.sort_values(by='loan_amount',ascending=False)

In [ ]:
m = folium.Map(location=[40, -95], zoom_start=4, prefer_canvas=True)

folium.Choropleth(
    geo_data=tracts,
    name="choropleth",
    data=med_loan_value,
    columns=["census_string", "loan_amount"],
    key_on="feature.properties.GEOID",
    fill_color="PuBu",
    fill_opacity=0.7,
    nan_fill_opacity = 0,
    line_opacity=.01,
    legend_name="Loan Amount",
    bins=[0,250_000,500_000,1_000_000,2_000_000,5_050_000]
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
applications = applications[applications['rate_spread'] != 'Exempt']

In [ ]:
applications['rate_spread'] = applications['rate_spread'].astype('float')

In [ ]:
spreads = applications.groupby('census_string')['rate_spread'].median()

In [ ]:
m = folium.Map(location=[40, -95], zoom_start=4, prefer_canvas=True)

folium.Choropleth(
    geo_data=tracts,
    name="choropleth",
    data=spreads,
    columns=["census_string", "rate_spread"],
    key_on="feature.properties.GEOID",
    fill_color="PuBu",
    fill_opacity=0.7,
    nan_fill_opacity = 0,
    line_opacity=.01,
    legend_name="Loan Amount",
    bins=[-5,-1,-0.5,0,0.5,1,2,3,5,10]
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
applications[applications['census_string'] == '36081056100']

In [ ]:
applications['census_string'].value_counts()